# NanoEventsFactory example

Load a single file from x-cache (using redirector_ucsd) or the grid (using redirector_fnal), and play with the events.


In [ ]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.analysis_tools import Weights, PackedSelection

from Tools.config_helpers import redirector_fnal, redirector_ucsd
from Tools.nano_mapping import make_fileset
from Tools.helpers import get_samples
from Tools.gen import get_charge_parent, find_first_parent

import awkward as ak
import numpy as np

#amples = get_samples(2018)

fileset = make_fileset(['Data'], 2018, redirector=redirector_ucsd, small=True)

#f_in = '/store/mc/RunIIAutumn18NanoAODv7/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/60000/022107FA-F567-1B44-B139-A18ADC996FCF.root'

# load a subset of events
n_max = 500000
events = NanoEventsFactory.from_root(
    fileset[list(fileset.keys())[0]][0],
    schemaclass = NanoAODSchema,
    #entry_stop = n_max,
).events()

In [ ]:
fileset[list(fileset.keys())[0]][1]

In [ ]:
#events.event

In [ ]:
#ev = events[events.event==33539380]

In [ ]:
#print(events[events.event==33539380].Electron.pt, events[events.event==33539380].Electron.eta, events[events.event==33539380].Electron.phi, events[events.event==33539380].Electron.charge,)

In [ ]:
import time

from Tools.objects import Collections

start_time = time.time()
electron = Collections(events, 'Electron', 'tightFCNC', verbose=1).get()
#muons = Collections(events, 'Muon', 'tight', verbose=1).get()

delta_time = time.time()-start_time

print ("\nTook %s seconds"%delta_time)

In [ ]:
from Tools.objects import *
electron = electron[(electron.pt > 20) & (np.abs(electron.eta) < 2.4)]

In [ ]:
leading_electron_idx = ak.singletons(ak.argmax(electron.pt, axis=1))
leading_electron = electron[(leading_electron_idx)]
leading_electron = leading_electron[(leading_electron.pt > 25)]

In [ ]:
dielectron = choose(electron, 2)
dielectron_mass = (dielectron['0']+dielectron['1']).mass


In [ ]:
#non_flip_events = events[((ak.num(electron, axis=1)==2)&(ak.sum(electron.charge, axis=1)!=0)&(ak.num(flipped_electron, axis=1)==0))]
print((ak.min(np.abs(dielectron_mass-91.2), axis=1) < 15))
print((np.abs(dielectron_mass-91.2) < 15))

In [ ]:
SSelectron = (ak.sum(electron.charge, axis=1) != 0) & (ak.num(electron)==2)
SSelectron

You can do anything that you're usually doing inside the processor here as well, so this is very useful for debugging

In [ ]:
matched_electrons = electrons[electrons.genPartIdx>=0]
sum(ak.num(matched_electrons, axis=1))

In [ ]:
f_in = '/store/mc/RunIIAutumn18NanoAODv7/QCD_Pt-120to170_MuEnrichedPt5_TuneCP5_13TeV_pythia8/NANOAODSIM/Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/70000/DE335891-829A-B943-99BE-E5A179F5F3EB.root'

events = NanoEventsFactory.from_root(
    redirector_ucsd + f_in,
    schemaclass = NanoAODSchema,
    entry_stop = 9999999).events()

muons = Collections(events, 'Muon', 'tight', verbose=1).get()

single_mu_ev = events[ak.num(muons)>0]

event_list_tight = single_mu_ev.event
event_list_tight

In [ ]:
import numpy as np
event_selector = (np.zeros_like(events.MET.pt) == 1)

In [ ]:
my_events = [29552, 12024433]

In [ ]:
for ev in my_events:
    event_selector = (event_selector | (events.event == ev))

In [ ]:
def get_charge_parent(particle):
    parent = find_first_parent(particle)
    charge = ak.zeros_like(parent)
    one = [[-11, -13, -15, -17, 24, 37], 1]
    minus_one = [[11, 13, 15, 17, -24, -37], -1]
    two_thirds = [[2, 4, 6, 8], 2/3]
    minus_two_thirds = [[-2, -4, -6, -8], -2/3]
    minus_one_third = [[1, 3, 5, 7], -1/3]
    one_third = [[-1, -3, -5, -7], 1/3]
    zero = [[12, 14, 16, 18, 9, 21, 22, 23, 25], 0]
    
    charge_pairs = [one, minus_one, two_thirds, minus_two_thirds, minus_one_third, zero]
    
    for pair in charge_pairs:
        for ID in pair[0]:
            charge = (parent == ID)*ak.ones_like(parent)*pair[1] + (~(parent == ID))*charge
            
    return charge

In [ ]:
mu_df = ak.to_pandas(ak.flatten(muons[event_selector][fields_to_show]))
mu_df

In [ ]:
ev_df = ak.to_pandas(events.MET[event_selector])
ev_df

In [ ]:
import pandas as pd
pd.concat([mu_df, ev_df], axis=1, )

## Some charge flip work

In [ ]:
!wget http://uaf-8.t2.ucsd.edu/~ewallace/chargeflipfull2016.pkl.gz

In [ ]:
from Tools.helpers import yahist_2D_lookup
import gzip
import pickle
 
class charge_flip:
    def __init__(self, path):
        self.path = path
        with gzip.open(self.path) as fin:
            self.ratio= pickle.load(fin)
    
    def flip_ratio(self, lepton1, lepton2):
        """takes a dilepton event and weights it based on the 
        odds that one of the leptons has a charge flip"""

        flip1 = yahist_2D_lookup(self.ratio, lepton1.pt, abs(lepton1.eta))
        flip2 = yahist_2D_lookup(self.ratio, lepton2.pt, abs(lepton2.eta))

        flip_rate1 = (ak.prod(flip1, axis = 1) * ak.prod(1/(1-flip1), axis = 1) * ak.prod(1-flip2/(1-flip2), axis = 1)) + (ak.prod(flip2, axis = 1) * ak.prod(1/(1-flip2), axis = 1) * ak.prod(1-flip1/(1-flip1), axis = 1))

        return flip_rate1
    
    def flip_weight(self, electron):

        f_1 = yahist_2D_lookup(self.ratio, electron.pt[:,0:1], abs(electron.eta[:,0:1]))
        f_2 = yahist_2D_lookup(self.ratio, electron.pt[:,1:2], abs(electron.eta[:,1:2]))

        # I'm using ak.prod and ak.sum to replace empty arrays by 1 and 0, respectively
        weight = ak.sum(f_1/(1-f_1), axis=1)*ak.prod(1-f_2/(1-f_2), axis=1) + ak.sum(f_2/(1-f_2), axis=1)*ak.prod(1-f_1/(1-f_1), axis=1)

        return weight

In [ ]:
cf = charge_flip('chargeflipfull2016.pkl.gz')

In [ ]:
electrons.matched_gen.parent.pdgId

In [ ]:
gen_matched_electrons = electrons[( (electrons.genPartIdx >= 0) & (abs(electrons.matched_gen.pdgId)==11) )]

In [ ]:
gen_matched_electrons.eta.tolist()

In [ ]:
is_flipped =( (gen_matched_electrons.matched_gen.pdgId*(-1) == gen_matched_electrons.pdgId) & (abs(gen_matched_electrons.pdgId) == 11) )

In [ ]:
flipped_electrons = gen_matched_electrons[is_flipped]

In [ ]:
flipped_electrons = flipped_electrons[(ak.fill_none(flipped_electrons.pt, 0)>0)]

In [ ]:
flipped_electrons.pt.tolist()